In [1]:
import numpy as np
import xarray as xr
import pandas as pd

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## To compute the linear regression we need data files:
- CMIP5_time_series.nc
- CMIP5_time_series_smoothed.nc
- CMIP5_time_series_smoothed_10yr.nc
- CMIP6_time_series.nc
- CMIP6_time_series_smoothed.nc
- CMIP6_time_series_smoothed_10yr.nc

In the next cell you can choose either the smoothed or unsmoothed time series.

In [2]:
data_choice = 'Smoothed'

if data_choice == 'Unsmoothed':
    dsCMIP5 = xr.open_dataset('Data/CMIP5_time_series.nc')
    dsCMIP6 = xr.open_dataset('Data/CMIP6_time_series.nc')

elif data_choice == 'Smoothed':
    dsCMIP5 = xr.open_dataset('Data/CMIP5_time_series_smoothed.nc')
    dsCMIP6 = xr.open_dataset('Data/CMIP6_time_series_smoothed.nc')

elif data_choice == 'Smoothed_10yr':
    dsCMIP5 = xr.open_dataset('Data/CMIP5_time_series_smoothed_10yr.nc')
    dsCMIP6 = xr.open_dataset('Data/CMIP6_time_series_smoothed_10yr.nc')

else:
    print('data_choice is not specified correctly. Choose `Smoothed`, `Smoothed_10yr` or `Unsmoothed`')

In [4]:
dsCMIP5_sce0 = xr.concat([dsCMIP5.isel(scenario=0),dsCMIP5.isel(scenario=1)*np.nan, dsCMIP5.isel(scenario=2)*np.nan],dim='scenario')
dsCMIP5_sce1 = xr.concat([dsCMIP5.isel(scenario=0)*np.nan,dsCMIP5.isel(scenario=1), dsCMIP5.isel(scenario=2)*np.nan],dim='scenario')
dsCMIP5_sce2 = xr.concat([dsCMIP5.isel(scenario=0)*np.nan,dsCMIP5.isel(scenario=1)*np.nan, dsCMIP5.isel(scenario=2)],dim='scenario')

In [5]:
dsCMIP6_sce0 = xr.concat([dsCMIP6.isel(scenario=0),dsCMIP6.isel(scenario=1)*np.nan, dsCMIP6.isel(scenario=2)*np.nan],dim='scenario')
dsCMIP6_sce1 = xr.concat([dsCMIP6.isel(scenario=0)*np.nan,dsCMIP6.isel(scenario=1), dsCMIP6.isel(scenario=2)*np.nan],dim='scenario')
dsCMIP6_sce2 = xr.concat([dsCMIP6.isel(scenario=0)*np.nan,dsCMIP6.isel(scenario=1)*np.nan, dsCMIP6.isel(scenario=2)],dim='scenario')

## Functions for linear regression:
- lin_reg_multi(varx, vary)

In [6]:
def select_models(ds, varx1, vary, check, varx2=None):
    '''
    Check is either 'model', or 'scenario'
    Find models for which both variables (varx1, vary) are available
    Input is an xarray dataset (CMIP5ds, CMIP5ds_LF, CMIP6ds, CMIP6ds_LF). 
    If you want to check the scenarios per model, already select the model in the input: eg. CMIP5ds.sel(model=mod)
    It returns the model/sce names of models/sces that have data for all three variables and returns the new dataset.
    '''
    x1 = ds[varx1].dropna(check,'all')[check].values
    y = ds[vary].dropna(check,'all')[check].values
    
    if varx2 == None:
        check_list = np.sort(list(set(x1)&set(y)))

    if varx2 is not None:
        x2 = ds[varx2].dropna(check,'all')[check].values
        check_list = np.sort(list(set(x1)&set(x2)&set(y)))
    
    if check == 'model':
        ds_new = ds.sel(model=check_list)
        
    elif check == 'scenario':
        ds_new = ds.sel(scenario=check_list)
    
    return check_list, ds_new

In [7]:
#%% Multi-linear regression
def lin_reg_multi(varx, vary):
    regr = linear_model.LinearRegression()

    varx = varx.dropna()
    vary = vary.dropna()

    regr.fit(varx, vary)
    
    vary_pred = regr.predict(varx)

    mse = mean_squared_error(vary, vary_pred)
    r2 = r2_score(vary, vary_pred)
    slope = regr.coef_
    intercept = regr.intercept_
    
    return vary_pred, mse, r2, slope, intercept

In [8]:
def lin_fit(ds, varx, mip):    
    ds_new = select_models(ds, varx, 'zos', 'model')[1]
    
    nan_array_sce = np.ones(95)*np.nan
    mods, sces = [],[]
    dfs = []
    mses, r2s, slope_varx, intercepts = [],[],[],[]
    
    for j, mod in enumerate(ds_new.model.values):
        
        if mod == 'GFDL-CM3':           # model does not have zos available for same scenarios as one of the regressors
            pass 
        else:
            sces_in_mod = select_models(ds_new.sel(model=mod), varx, 'zos', 'scenario')[0]
            ds_mod = ds_new.sel(model=mod)
            mods.append(mod)           # append model name
            sces.append(sces_in_mod)   # append scenarios available for this model
        
            if mip == 'CMIP5': 
                end_hist = 2005
                nan_num = 95
                sce_names = ['rcp26','rcp45','rcp85']

            elif mip == 'CMIP6':
                end_hist = 2014
                nan_num = 86
                sce_names = ['ssp126','ssp245','ssp585']

            ds_hist = ds_mod.sel(time=slice(1900,end_hist),scenario=sces_in_mod[0])  # select historical period for one sce
            ds_sces = ds_mod.sel(time=slice(end_hist,2100))                          # select future period for all sces

            DSL_mod, varx_mod = ds_hist.zos.values, ds_hist[varx].values      
        
            for sce in sce_names:        
                ds_sce = ds_sces.sel(scenario=sce)
            
                zos = ds_sce.zos.values
                varx_list = ds_sce[varx].values
            
                if(np.isnan(zos).any()) or (np.isnan(varx_list).any()):
                    DSL_mod = np.append(DSL_mod, np.ones(nan_num)*np.nan)
                    varx_mod = np.append(varx_mod, np.ones(nan_num)*np.nan)
                else:
                    DSL_mod = np.append(DSL_mod, zos)
                    varx_mod = np.append(varx_mod, varx_list)
            
            # Store total data in dataframe for each model
            d = {'DSL': DSL_mod, 'varx': varx_mod}
            df = pd.DataFrame(data=d)    
            dfs.append(df)
         
            # Compute regression for this model    
            X = df[['varx']]
            Y = df[['DSL']]
        
            #print(df)
            linreg = lin_reg_multi(X,Y)
            mse, r2, slope, intercept = linreg[1], linreg[2], linreg[3], linreg[4]
        
            mses.append(mse)
            r2s.append(r2)
            slope_varx.append(slope[0][0])
            intercepts.append(intercept[0])

    # Construct dataframe to store parameter values
    d = {'model': mods, 'sces':sces, 'r2-score':r2s, 'alpha': intercepts, 'beta':slope_varx, 'mse': mses}
    df_params = pd.DataFrame(data=d).set_index('model').round(4)
            
    return dfs, df_params

In [9]:
def multi_lin_fit(ds, varx1, varx2, mip):
    
    ds_new = select_models(ds, varx1, 'zos','model',varx2)[1]
    
    nan_array_sce = np.ones(95)*np.nan
    mods, sces = [],[]
    dfs = []
    mses, r2s, slope_varx1, slope_varx2, intercepts = [],[],[],[],[]
    
    for j, mod in enumerate(ds_new.model.values):
        sces_in_mod = select_models(ds_new.sel(model=mod), varx1, 'zos', 'scenario', varx2)[0]
        ds_mod = ds_new.sel(model=mod)
        mods.append(mod)           # append model name
        sces.append(sces_in_mod)   # append scenarios available for this model
        
        if mip == 'CMIP5': 
            end_hist = 2005
            nan_num = 95
            sce_names = ['rcp26','rcp45','rcp85']

        elif mip == 'CMIP6':
            end_hist = 2014
            nan_num = 86
            sce_names = ['ssp126','ssp245','ssp585']
                
        ds_hist = ds_mod.sel(time=slice(1900,end_hist),scenario=sces_in_mod[0])  # select historical period for one sce
        ds_sces = ds_mod.sel(time=slice(end_hist,2100))                          # select future period for all sces

        DSL_mod, varx1_mod, varx2_mod = ds_hist.zos.values, ds_hist[varx1].values, ds_hist[varx2].values        
        
            
        for sce in sce_names:        
            ds_sce = ds_sces.sel(scenario=sce)
            
            zos = ds_sce.zos.values
            varx1_list = ds_sce[varx1].values
            varx2_list = ds_sce[varx2].values
            
            if(np.isnan(zos).any()) or (np.isnan(varx1_list).any()) or (np.isnan(varx2_list).any()):
                DSL_mod = np.append(DSL_mod, np.ones(nan_num)*np.nan)
                varx1_mod = np.append(varx1_mod, np.ones(nan_num)*np.nan)
                varx2_mod = np.append(varx2_mod, np.ones(nan_num)*np.nan)
            else:
                DSL_mod = np.append(DSL_mod, zos)
                varx1_mod = np.append(varx1_mod, varx1_list)
                varx2_mod = np.append(varx2_mod, varx2_list)
            
        # Store total data in dataframe for each model
        d = {'DSL': DSL_mod, 'varx1': varx1_mod, 'varx2': varx2_mod}
        df = pd.DataFrame(data=d)    
        dfs.append(df)
         
        # Compute regression for this model    
        X = df[['varx1','varx2']] 
        Y = df[['DSL']] 
        
        #print(df)
        linreg = lin_reg_multi(X,Y)
        mse, r2, slope, intercept = linreg[1], linreg[2], linreg[3], linreg[4]
        
        mses.append(mse)
        r2s.append(r2)
        slope_varx1.append(slope[0][0])
        slope_varx2.append(slope[0][1])
        intercepts.append(intercept[0])
    
        
    # Construct dataframe to store parameter values
    d = {'model': mods, 'sces':sces, 'r2-score':r2s,'alpha': intercepts, 'beta1':slope_varx1, 'beta2': slope_varx2, 'mse': mses}
    df_params = pd.DataFrame(data=d).set_index('model').round(4)
    
            
    return dfs, df_params

## Single regression

In [15]:
LR_CMIP5_GSAT_ts,   LR_CMIP5_GSAT    = lin_fit(dsCMIP5, 'GSAT',   'CMIP5')
LR_CMIP5_GMTSL_ts,  LR_CMIP5_GMTSL   = lin_fit(dsCMIP5, 'GMTSL',  'CMIP5')
LR_CMIP5_AMOC35_ts, LR_CMIP5_AMOC35  = lin_fit(dsCMIP5, 'AMOC35', 'CMIP5')
LR_CMIP5_AMOC26_ts, LR_CMIP5_AMOC26  = lin_fit(dsCMIP5, 'AMOC26', 'CMIP5')

LR_CMIP6_GSAT_ts,   LR_CMIP6_GSAT    = lin_fit(dsCMIP6, 'GSAT',   'CMIP6')
LR_CMIP6_GMTSL_ts,  LR_CMIP6_GMTSL   = lin_fit(dsCMIP6, 'GMTSL',  'CMIP6')
LR_CMIP6_AMOC35_ts, LR_CMIP6_AMOC35  = lin_fit(dsCMIP6, 'AMOC35', 'CMIP6')
LR_CMIP6_AMOC26_ts, LR_CMIP6_AMOC26  = lin_fit(dsCMIP6, 'AMOC26', 'CMIP6')

In [16]:
LR_CMIP5_GSAT_ts_sce0,   LR_CMIP5_GSAT_sce0    = lin_fit(dsCMIP5_sce0, 'GSAT',   'CMIP5')
LR_CMIP5_GSAT_ts_sce1,   LR_CMIP5_GSAT_sce1    = lin_fit(dsCMIP5_sce1, 'GSAT',   'CMIP5')
LR_CMIP5_GSAT_ts_sce2,   LR_CMIP5_GSAT_sce2    = lin_fit(dsCMIP5_sce2, 'GSAT',   'CMIP5')

LR_CMIP5_GMTSL_ts_sce0,   LR_CMIP5_GMTSL_sce0    = lin_fit(dsCMIP5_sce0, 'GMTSL',   'CMIP5')
LR_CMIP5_GMTSL_ts_sce1,   LR_CMIP5_GMTSL_sce1    = lin_fit(dsCMIP5_sce1, 'GMTSL',   'CMIP5')
LR_CMIP5_GMTSL_ts_sce2,   LR_CMIP5_GMTSL_sce2    = lin_fit(dsCMIP5_sce2, 'GMTSL',   'CMIP5')

LR_CMIP5_AMOC35_ts_sce0,   LR_CMIP5_AMOC35_sce0    = lin_fit(dsCMIP5_sce0, 'AMOC35',   'CMIP5')
LR_CMIP5_AMOC35_ts_sce1,   LR_CMIP5_AMOC35_sce1    = lin_fit(dsCMIP5_sce1, 'AMOC35',   'CMIP5')
LR_CMIP5_AMOC35_ts_sce2,   LR_CMIP5_AMOC35_sce2    = lin_fit(dsCMIP5_sce2, 'AMOC35',   'CMIP5')


LR_CMIP6_GSAT_ts_sce0,   LR_CMIP6_GSAT_sce0    = lin_fit(dsCMIP6_sce0, 'GSAT',   'CMIP6')
LR_CMIP6_GSAT_ts_sce1,   LR_CMIP6_GSAT_sce1    = lin_fit(dsCMIP6_sce1, 'GSAT',   'CMIP6')
LR_CMIP6_GSAT_ts_sce2,   LR_CMIP6_GSAT_sce2    = lin_fit(dsCMIP6_sce2, 'GSAT',   'CMIP6')

LR_CMIP6_GMTSL_ts_sce0,   LR_CMIP6_GMTSL_sce0    = lin_fit(dsCMIP6_sce0, 'GMTSL',   'CMIP6')
LR_CMIP6_GMTSL_ts_sce1,   LR_CMIP6_GMTSL_sce1    = lin_fit(dsCMIP6_sce1, 'GMTSL',   'CMIP6')
LR_CMIP6_GMTSL_ts_sce2,   LR_CMIP6_GMTSL_sce2    = lin_fit(dsCMIP6_sce2, 'GMTSL',   'CMIP6')

LR_CMIP6_AMOC35_ts_sce0,   LR_CMIP6_AMOC35_sce0    = lin_fit(dsCMIP6_sce0, 'AMOC35',   'CMIP6')
LR_CMIP6_AMOC35_ts_sce1,   LR_CMIP6_AMOC35_sce1    = lin_fit(dsCMIP6_sce1, 'AMOC35',   'CMIP6')
LR_CMIP6_AMOC35_ts_sce2,   LR_CMIP6_AMOC35_sce2    = lin_fit(dsCMIP6_sce2, 'AMOC35',   'CMIP6')

In [17]:
print('CMIP5 GSAT')
print('sceal rmse:',np.round(LR_CMIP5_GSAT['mse'].mean(),2),      'b1:', np.round(LR_CMIP5_GSAT['beta'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP5_GSAT_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_sce0['beta'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP5_GSAT_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_sce1['beta'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP5_GSAT_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_sce2['beta'].mean(),2))
print('CMIP6 GSAT')
print('sceal rmse:',np.round(LR_CMIP6_GSAT['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT['beta'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP6_GSAT_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_sce0['beta'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP6_GSAT_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_sce1['beta'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP6_GSAT_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_sce2['beta'].mean(),2))


CMIP5 GSAT
sceal rmse: 3.1 b1: 3.62
sce26 rmse: 2.28 b1: 3.47
sce45 rmse: 2.11 b1: 3.62
sce85 rmse: 2.31 b1: 3.57
CMIP6 GSAT
sceal rmse: 10.19 b1: 4.58
sce26 rmse: 6.53 b1: 6.05
sce45 rmse: 4.06 b1: 5.42
sce85 rmse: 4.35 b1: 4.23


In [18]:
print('CMIP5 GMTSL')
print('sceal rmse:',np.round(LR_CMIP5_GMTSL['mse'].mean(),2),      'b1:', np.round(LR_CMIP5_GMTSL['beta'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP5_GMTSL_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GMTSL_sce0['beta'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP5_GMTSL_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GMTSL_sce1['beta'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP5_GMTSL_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GMTSL_sce2['beta'].mean(),2))
print('CMIP6 GTMSL')
print('sceal rmse:',np.round(LR_CMIP6_GMTSL['mse'].mean(),2),      'b1:', np.round(LR_CMIP6_GMTSL['beta'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP6_GMTSL_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GMTSL_sce0['beta'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP6_GMTSL_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GMTSL_sce1['beta'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP6_GMTSL_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GMTSL_sce2['beta'].mean(),2))


CMIP5 GMTSL
sceal rmse: 2.82 b1: 0.54
sce26 rmse: 2.25 b1: 0.48
sce45 rmse: 2.1 b1: 0.54
sce85 rmse: 2.27 b1: 0.55
CMIP6 GTMSL
sceal rmse: 7.17 b1: 0.87
sce26 rmse: 4.03 b1: 0.99
sce45 rmse: 4.21 b1: 0.96
sce85 rmse: 5.88 b1: 0.86


In [19]:
print('CMIP5 AMOC35')
print('sceal rmse:',np.round(LR_CMIP5_AMOC35['mse'].mean(),2),      'b1:', np.round(LR_CMIP5_AMOC35['beta'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP5_AMOC35_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_AMOC35_sce0['beta'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP5_AMOC35_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_AMOC35_sce1['beta'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP5_AMOC35_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_AMOC35_sce2['beta'].mean(),2))
print('CMIP6 AMOC35')
print('sceal rmse:',np.round(LR_CMIP6_AMOC35['mse'].mean(),2),      'b1:', np.round(LR_CMIP6_AMOC35['beta'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP6_AMOC35_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_AMOC35_sce0['beta'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP6_AMOC35_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_AMOC35_sce1['beta'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP6_AMOC35_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_AMOC35_sce2['beta'].mean(),2))


CMIP5 AMOC35
sceal rmse: 5.43 b1: -2.44
sce26 rmse: 3.11 b1: -1.77
sce45 rmse: 4.25 b1: -2.35
sce85 rmse: 5.58 b1: -2.53
CMIP6 AMOC35
sceal rmse: 7.12 b1: -2.11
sce26 rmse: 6.86 b1: -2.08
sce45 rmse: 6.86 b1: -2.16
sce85 rmse: 7.68 b1: -2.15


## Multiple regression

In [20]:
LR_CMIP5_GSAT_GMTSL     = multi_lin_fit(dsCMIP5, 'GSAT',  'GMTSL',  'CMIP5')[1]
LR_CMIP5_GSAT_AMOC35    = multi_lin_fit(dsCMIP5, 'GSAT',  'AMOC35', 'CMIP5')[1]
LR_CMIP5_GSAT_AMOC26    = multi_lin_fit(dsCMIP5, 'GSAT',  'AMOC26', 'CMIP5')[1]
LR_CMIP5_GMTSL_AMOC35   = multi_lin_fit(dsCMIP5, 'GMTSL', 'AMOC35', 'CMIP5')[1]
LR_CMIP5_GMTSL_AMOC26   = multi_lin_fit(dsCMIP5, 'GMTSL', 'AMOC26', 'CMIP5')[1]

LR_CMIP6_GSAT_GMTSL     = multi_lin_fit(dsCMIP6, 'GSAT',  'GMTSL',  'CMIP6')[1]
LR_CMIP6_GSAT_AMOC35    = multi_lin_fit(dsCMIP6, 'GSAT',  'AMOC35', 'CMIP6')[1]
LR_CMIP6_GSAT_AMOC26    = multi_lin_fit(dsCMIP6, 'GSAT',  'AMOC26', 'CMIP6')[1]
LR_CMIP6_GMTSL_AMOC35   = multi_lin_fit(dsCMIP6, 'GMTSL', 'AMOC35', 'CMIP6')[1]
LR_CMIP6_GMTSL_AMOC26   = multi_lin_fit(dsCMIP6, 'GMTSL', 'AMOC26', 'CMIP6')[1]

In [21]:
LR_CMIP5_GSAT_GMTSL_sce0     = multi_lin_fit(dsCMIP5_sce0, 'GSAT',  'GMTSL',  'CMIP5')[1]
LR_CMIP5_GSAT_AMOC35_sce0    = multi_lin_fit(dsCMIP5_sce0, 'GSAT',  'AMOC35', 'CMIP5')[1]
LR_CMIP5_GMTSL_AMOC35_sce0   = multi_lin_fit(dsCMIP5_sce0, 'GMTSL', 'AMOC35', 'CMIP5')[1]

LR_CMIP5_GSAT_GMTSL_sce1     = multi_lin_fit(dsCMIP5_sce1, 'GSAT',  'GMTSL',  'CMIP5')[1]
LR_CMIP5_GSAT_AMOC35_sce1    = multi_lin_fit(dsCMIP5_sce1, 'GSAT',  'AMOC35', 'CMIP5')[1]
LR_CMIP5_GMTSL_AMOC35_sce1   = multi_lin_fit(dsCMIP5_sce1, 'GMTSL', 'AMOC35', 'CMIP5')[1]

LR_CMIP5_GSAT_GMTSL_sce2     = multi_lin_fit(dsCMIP5_sce2, 'GSAT',  'GMTSL',  'CMIP5')[1]
LR_CMIP5_GSAT_AMOC35_sce2    = multi_lin_fit(dsCMIP5_sce2, 'GSAT',  'AMOC35', 'CMIP5')[1]
LR_CMIP5_GMTSL_AMOC35_sce2   = multi_lin_fit(dsCMIP5_sce2, 'GMTSL', 'AMOC35', 'CMIP5')[1]

LR_CMIP6_GSAT_GMTSL_sce0     = multi_lin_fit(dsCMIP6_sce0, 'GSAT',  'GMTSL',  'CMIP6')[1]
LR_CMIP6_GSAT_AMOC35_sce0    = multi_lin_fit(dsCMIP6_sce0, 'GSAT',  'AMOC35', 'CMIP6')[1]
LR_CMIP6_GMTSL_AMOC35_sce0   = multi_lin_fit(dsCMIP6_sce0, 'GMTSL', 'AMOC35', 'CMIP6')[1]

LR_CMIP6_GSAT_GMTSL_sce1     = multi_lin_fit(dsCMIP6_sce1, 'GSAT',  'GMTSL',  'CMIP6')[1]
LR_CMIP6_GSAT_AMOC35_sce1    = multi_lin_fit(dsCMIP6_sce1, 'GSAT',  'AMOC35', 'CMIP6')[1]
LR_CMIP6_GMTSL_AMOC35_sce1   = multi_lin_fit(dsCMIP6_sce1, 'GMTSL', 'AMOC35', 'CMIP6')[1]

LR_CMIP6_GSAT_GMTSL_sce2     = multi_lin_fit(dsCMIP6_sce2, 'GSAT',  'GMTSL',  'CMIP6')[1]
LR_CMIP6_GSAT_AMOC35_sce2    = multi_lin_fit(dsCMIP6_sce2, 'GSAT',  'AMOC35', 'CMIP6')[1]
LR_CMIP6_GMTSL_AMOC35_sce2   = multi_lin_fit(dsCMIP6_sce2, 'GMTSL', 'AMOC35', 'CMIP6')[1]


In [22]:
print('CMIP5 GSAT_GMTSL')
print('sceal rmse:',np.round(LR_CMIP5_GSAT_GMTSL['mse'].mean(),2),      'b1:', np.round(LR_CMIP5_GSAT_GMTSL['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_GMTSL['beta2'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP5_GSAT_GMTSL_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_GMTSL_sce0['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_GMTSL_sce0['beta2'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP5_GSAT_GMTSL_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_GMTSL_sce1['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_GMTSL_sce1['beta2'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP5_GSAT_GMTSL_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_GMTSL_sce2['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_GMTSL_sce2['beta2'].mean(),2))
print('CMIP6 GSAT_GMTSL')
print('sceal rmse:',np.round(LR_CMIP6_GSAT_GMTSL['mse'].mean(),2),      'b1:', np.round(LR_CMIP6_GSAT_GMTSL['beta1'].mean(),2),      'b2:', np.round(LR_CMIP6_GSAT_GMTSL['beta2'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP6_GSAT_GMTSL_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_GMTSL_sce0['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GSAT_GMTSL_sce0['beta2'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP6_GSAT_GMTSL_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_GMTSL_sce1['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GSAT_GMTSL_sce1['beta2'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP6_GSAT_GMTSL_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_GMTSL_sce2['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GSAT_GMTSL_sce2['beta2'].mean(),2))


CMIP5 GSAT_GMTSL
sceal rmse: 2.36 b1: 2.08 b2: 0.29
sce26 rmse: 1.65 b1: 2.44 b2: 0.21
sce45 rmse: 1.55 b1: 2.12 b2: 0.28
sce85 rmse: 1.64 b1: 3.28 b2: 0.1
CMIP6 GSAT_GMTSL
sceal rmse: 6.25 b1: 1.66 b2: 0.62
sce26 rmse: 2.91 b1: 2.98 b2: 0.62
sce45 rmse: 2.92 b1: 4.02 b2: 0.41
sce85 rmse: 3.38 b1: 6.96 b2: -0.24


In [23]:
print('CMIP5 GSAT_AMOC35')
print('sceal rmse:',np.round(LR_CMIP5_GSAT_AMOC35['mse'].mean(),2),      'b1:', np.round(LR_CMIP5_GSAT_AMOC35['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_AMOC35['beta2'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP5_GSAT_AMOC35_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_AMOC35_sce0['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_AMOC35_sce0['beta2'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP5_GSAT_AMOC35_sce1['mse'].mean(),2), 'b1:',                  np.round(LR_CMIP5_GSAT_AMOC35_sce1['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_AMOC35_sce1['beta2'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP5_GSAT_AMOC35_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GSAT_AMOC35_sce2['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GSAT_AMOC35_sce2['beta2'].mean(),2))
print('CMIP6 GSAT_AMOC35')
print('sceal rmse:',np.round(LR_CMIP6_GSAT_AMOC35['mse'].mean(),2),      'b1:', np.round(LR_CMIP6_GSAT_AMOC35['beta1'].mean(),2),      'b2:', np.round(LR_CMIP6_GSAT_AMOC35['beta2'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP6_GSAT_AMOC35_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_AMOC35_sce0['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GSAT_AMOC35_sce0['beta2'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP6_GSAT_AMOC35_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_AMOC35_sce1['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GSAT_AMOC35_sce1['beta2'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP6_GSAT_AMOC35_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GSAT_AMOC35_sce2['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GSAT_AMOC35_sce2['beta2'].mean(),2))


CMIP5 GSAT_AMOC35
sceal rmse: 2.58 b1: 2.75 b2: -0.64
sce26 rmse: 1.75 b1: 2.3 b2: -0.69
sce45 rmse: 1.75 b1: 3.04 b2: -0.6
sce85 rmse: 1.8 b1: 3.48 b2: -0.31
CMIP6 GSAT_AMOC35
sceal rmse: 4.01 b1: 1.93 b2: -1.29
sce26 rmse: 2.56 b1: 3.18 b2: -1.01
sce45 rmse: 2.31 b1: 3.5 b2: -0.8
sce85 rmse: 2.83 b1: 2.84 b2: -0.79


In [24]:
print('CMIP5 GSAT_AMOC35')
print('sceal rmse:',np.round(LR_CMIP5_GMTSL_AMOC35['mse'].mean(),2),      'b1:', np.round(LR_CMIP5_GMTSL_AMOC35['beta1'].mean(),2),      'b2:', np.round(LR_CMIP5_GMTSL_AMOC35['beta2'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP5_GMTSL_AMOC35_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GMTSL_AMOC35_sce0['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GMTSL_AMOC35_sce0['beta2'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP5_GMTSL_AMOC35_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GMTSL_AMOC35_sce1['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GMTSL_AMOC35_sce1['beta2'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP5_GMTSL_AMOC35_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP5_GMTSL_AMOC35_sce2['beta1'].mean(),2), 'b2:', np.round(LR_CMIP5_GMTSL_AMOC35_sce2['beta2'].mean(),2))
print('CMIP6 GSAT_AMOC35')
print('sceal rmse:',np.round(LR_CMIP6_GMTSL_AMOC35['mse'].mean(),2),      'b1:', np.round(LR_CMIP6_GMTSL_AMOC35['beta1'].mean(),2),      'b2:', np.round(LR_CMIP6_GMTSL_AMOC35['beta2'].mean(),2))
print('sce26 rmse:',np.round(LR_CMIP6_GMTSL_AMOC35_sce0['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GMTSL_AMOC35_sce0['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GMTSL_AMOC35_sce0['beta2'].mean(),2))
print('sce45 rmse:',np.round(LR_CMIP6_GMTSL_AMOC35_sce1['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GMTSL_AMOC35_sce1['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GMTSL_AMOC35_sce1['beta2'].mean(),2))
print('sce85 rmse:',np.round(LR_CMIP6_GMTSL_AMOC35_sce2['mse'].mean(),2), 'b1:', np.round(LR_CMIP6_GMTSL_AMOC35_sce2['beta1'].mean(),2), 'b2:', np.round(LR_CMIP6_GMTSL_AMOC35_sce2['beta2'].mean(),2))


CMIP5 GSAT_AMOC35
sceal rmse: 2.32 b1: 0.4 b2: -0.82
sce26 rmse: 1.65 b1: 0.3 b2: -0.9
sce45 rmse: 1.7 b1: 0.42 b2: -0.78
sce85 rmse: 1.98 b1: 0.51 b2: -0.34
CMIP6 GSAT_AMOC35
sceal rmse: 4.42 b1: 0.41 b2: -1.37
sce26 rmse: 2.58 b1: 0.57 b2: -1.0
sce45 rmse: 2.52 b1: 0.55 b2: -1.09
sce85 rmse: 3.26 b1: 0.48 b2: -1.15
